In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:

drop_out_val = 0.05 # seting dropout as 5 %

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # CONVOLUTION LAYER 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=(3,3), padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.do1 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 2
        self.conv2 = nn.Conv2d(12, 24, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(24)
        self.do2 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 3
        self.conv3  = nn.Conv2d(24, 36, 3, padding=1)
        self.bn3    = nn.BatchNorm2d(36)
        self.do3    = nn.Dropout2d(drop_out_val)

        # TRANSITION LAYER 1
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv_t1 = nn.Conv2d(36, 12, 1)
        self.bn_t1 = nn.BatchNorm2d(12)
        self.do_t1 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 4
        self.conv4 = nn.Conv2d(12, 12, 3, padding = 1)
        self.bn4 = nn.BatchNorm2d(12)
        self.do4 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 5
        self.conv5  = nn.Conv2d(12, 24, 3, padding = 1)
        self.bn5    = nn.BatchNorm2d(24)
        self.do5    = nn.Dropout2d(drop_out_val)


        # TRANSITION LAYER 2
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv_t2  = nn.Conv2d(24, 12, 1)
        self.bn_t2    = nn.BatchNorm2d(12)
        self.do_t2    = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 6
        self.conv6 = nn.Conv2d(12, 12, 3, padding = 1)
        self.bn6 = nn.BatchNorm2d(12)
        self.do6 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 7
        self.conv7 = nn.Conv2d(12, 24, 3, padding = 1)

        self.gap = nn.AvgPool2d(7)
        self.linear1 = nn.Linear(24, 10)

    def forward(self, x):
        #BN after Relu
        x = self.pool1(self.do3(self.bn3(F.relu(self.conv3 (self.do2(self.bn2(F.relu(self.conv2  (self.do1(self.bn1(F.relu(self.conv1(x)))))))))))))
        x = self.pool2(self.do5(self.bn5(F.relu(self.conv5(self.do4(self.bn4(self.conv4((self.do_t1(self.bn_t1(F.relu(self.conv_t1(x)))))))))))))
        x = F.relu( self.conv7(self.do6(self.bn6(F.relu(self.conv6 (self.do_t2(self.bn_t2(F.relu(self.conv_t2(x))))))))))
        x = self.gap(x)
        x = x.view(-1, 24)
        x = self.linear1(x)

        return F.log_softmax(x,dim=1)       #log of softmax

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
       BatchNorm2d-2           [-1, 12, 28, 28]              24
         Dropout2d-3           [-1, 12, 28, 28]               0
            Conv2d-4           [-1, 24, 28, 28]           2,616
       BatchNorm2d-5           [-1, 24, 28, 28]              48
         Dropout2d-6           [-1, 24, 28, 28]               0
            Conv2d-7           [-1, 36, 28, 28]           7,812
       BatchNorm2d-8           [-1, 36, 28, 28]              72
         Dropout2d-9           [-1, 36, 28, 28]               0
        MaxPool2d-10           [-1, 36, 14, 14]               0
           Conv2d-11           [-1, 12, 14, 14]             444
      BatchNorm2d-12           [-1, 12, 14, 14]              24
        Dropout2d-13           [-1, 12, 14, 14]               0
           Conv2d-14           [-1, 12,

In [4]:

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                              transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                              transforms.Resize((28, 28)),
                              transforms.RandomRotation((-15., 15.), fill=0),
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,)),
                              ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9.91M/9.91M [00:00<00:00, 11.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 344kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.76MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.58MB/s]


In [5]:
from tqdm import tqdm

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()


def train(model, device, train_loader, optimizer, epoch):
    correct = 0
    processed = 0
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        correct += GetCorrectPredCount(output, target)
        processed += len(data)
        pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

for epoch in range(1, 20):
    print(f'Epoch : {epoch} \n')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()
    print('.......................................................................................................................................')

Epoch : 1 



Train: Loss=0.2132 Batch_id=468 Accuracy=62.54: 100%|██████████| 469/469 [00:30<00:00, 15.55it/s]



Test: Average loss: 0.1459, Accuracy: 9625/10000 (96.25%)

.......................................................................................................................................
Epoch : 2 



Train: Loss=0.2153 Batch_id=468 Accuracy=92.69: 100%|██████████| 469/469 [00:29<00:00, 15.74it/s]



Test: Average loss: 0.1151, Accuracy: 9673/10000 (96.73%)

.......................................................................................................................................
Epoch : 3 



...


Test: Average loss: 0.0205, Accuracy: 9929/10000 (99.29%)

.......................................................................................................................................
Epoch : 18 



Train: Loss=0.0281 Batch_id=468 Accuracy=98.20: 100%|██████████| 469/469 [00:27<00:00, 17.24it/s]



Test: Average loss: 0.0197, Accuracy: 9936/10000 (99.36%)

.......................................................................................................................................
Epoch : 19 



Train: Loss=0.0355 Batch_id=468 Accuracy=98.19: 100%|██████████| 469/469 [00:26<00:00, 17.59it/s]



Test: Average loss: 0.0200, Accuracy: 9930/10000 (99.30%)

.......................................................................................................................................
